In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import time
import matplotlib.pyplot as plt
import yfinance as yf
import eikon as ek
ek.set_app_key('85ce298d92be42c6947658adba5934d1418e0c9a')

In [3]:
# Shanghai SE Composite Index
rmkt = yf.download("000001.SS", start="2006-01-01", end="2025-01-01", interval="1mo")
rmkt = rmkt[['Close']].dropna()
rmkt.index = rmkt.index - pd.offsets.Day(1)
rmkt['rmkt'] = np.log(rmkt['Close'] / rmkt['Close'].shift(1))
rmkt = rmkt[['rmkt']].dropna()
rmkt.columns = rmkt.columns.get_level_values(0)
rmkt.columns.name = None
rmkt.index.name = 'Date'
rmkt.to_csv("rmkt.csv")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
2025-06-11 22:05:26,005 P[9484] [MainThread 7864] 
1 Failed download:
2025-06-11 22:05:26,009 P[9484] [MainThread 7864] ['000001.SS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [2]:
# China 10-Year Government Bond Yield
file_path = r"C:\Users\hp\Favorites\Desktop\Thesis\China 10 Y Govt Bond Yield.xlsx"
rf = pd.read_excel(file_path, skiprows=1, engine="openpyxl")

# Rename and format
rf.columns = ["Date", "rf"]
rf["Date"] = pd.to_datetime(rf["Date"])
rf.set_index("Date", inplace=True)

# Resample to month-end
rf = rf.resample('ME').last()

# Convert annual rate (%) to monthly log return
rf["rf"] = np.log(1 + rf["rf"] / 100) / 12

# Drop missing values if any
rf = rf.dropna()
print(rf)

                  rf
Date                
2002-08-31  0.002339
2002-09-30  0.002529
2002-10-31  0.002518
2002-11-30  0.002723
2002-12-31  0.002543
...              ...
2024-11-30  0.001694
2024-12-31  0.001388
2025-01-31  0.001366
2025-02-28  0.001472
2025-03-31  0.001593

[272 rows x 1 columns]


In [11]:
# SOE df
file_path = r"C:\Users\hp\Favorites\Desktop\Thesis\All Chinese SOEs.xlsx"  
soes1 = pd.read_excel(file_path, usecols="A", engine="openpyxl")  
soes1.columns = ["Tickers"]

soes1_data_ek = []

for ticker in soes1["Tickers"]:
    try:
        print(f"Fetching data for {ticker}...")
        data = ek.get_timeseries(ticker, fields="CLOSE", start_date="2012-01-01", end_date="2025-01-01", interval="monthly")
        data['Ticker'] = ticker
        soes1_data_ek.append(data)
        
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
soes1_data_ = pd.concat(soes1_data_ek)
soes1_data = soes1_data_.pivot_table(index='Date', columns='Ticker', values='CLOSE')
soes1_data = soes1_data.dropna(axis=1)
rsoes1 = np.log(soes1_data) - np.log(soes1_data.shift(1))
rsoes1 = rsoes1.dropna()
rsoes1.columns.name = None

rsoes1.to_csv("rsoes1.csv")

Fetching data for 601398.SS...
Fetching data for 600519.SS...
Fetching data for 601288.SS...
Fetching data for 600941.SS...
Fetching data for 601939.SS...
Fetching data for 601988.SS...
Fetching data for 601857.SS...
Fetching data for 601628.SS...
Fetching data for 600938.SS...
Fetching data for 601088.SS...
Fetching data for 601728.SS...
Fetching data for 600900.SS...
Fetching data for 600028.SS...
Fetching data for 000858.SZ...
Fetching data for 601658.SS...
Fetching data for 601899.SS...
Fetching data for 601998.SS...
Fetching data for 002415.SZ...
Fetching data for 600000.SS...
Fetching data for 601319.SS...
Fetching data for 601816.SS...
Fetching data for 600809.SS...
Fetching data for 002371.SZ...
Fetching data for 601668.SS...
Fetching data for 600309.SS...
Fetching data for 601818.SS...
Fetching data for 601919.SS...
Fetching data for 000568.SZ...
Fetching data for 601766.SS...
Fetching data for 601985.SS...
Fetching data for 601225.SS...
Fetching data for 600050.SS...
Fetching

2025-06-12 09:23:28,275 P[23016] [MainThread 6232] Error with 600705.SS: Invalid RIC
2025-06-12 09:23:28,276 P[23016] [MainThread 6232] 600705.SS: Invalid RIC | 


Error fetching data for 600705.SS: Error code -1 | 600705.SS: Invalid RIC | 
Fetching data for 600879.SS...
Fetching data for 000958.SZ...
Fetching data for 601880.SS...
Fetching data for 601990.SS...
Fetching data for 600298.SS...
Fetching data for 300024.SZ...
Fetching data for 000021.SZ...
Fetching data for 688538.SS...
Fetching data for 688122.SS...
Fetching data for 000883.SZ...
Fetching data for 600038.SS...
Fetching data for 600995.SS...
Fetching data for 688297.SS...
Fetching data for 002465.SZ...
Fetching data for 000733.SZ...
Fetching data for 601696.SS...
Fetching data for 600369.SS...
Fetching data for 002423.SZ...
Fetching data for 600816.SS...
Fetching data for 600497.SS...
Fetching data for 000027.SZ...
Fetching data for 600153.SS...
Fetching data for 600517.SS...
Fetching data for 000032.SZ...
Fetching data for 600282.SS...
Fetching data for 600871.SS...
Fetching data for 600765.SS...
Fetching data for 600299.SS...
Fetching data for 600901.SS...
Fetching data for 600498

In [14]:
file_path = r"C:\Users\hp\Favorites\Desktop\Thesis\rsoes1.csv"
rsoes1 = pd.read_csv(file_path)
rsoes1['Date'] = pd.to_datetime(rsoes1['Date'])
rsoes1.set_index('Date', inplace=True)

file_path = r"C:\Users\hp\Favorites\Desktop\Thesis\rmkt.csv"
rmkt = pd.read_csv(file_path)
rmkt['Date'] = pd.to_datetime(rmkt['Date'])
rmkt.set_index('Date', inplace=True)

# Merge all the data (SOEs, Market, Risk-Free Rate)
soes1_capm = pd.merge(pd.merge(rsoes1, rmkt, on="Date", how="inner"), rf, on="Date", how="inner")

soes1_capm_exc = soes1_capm.copy()
for col in soes1_capm.columns:
    if col != "rf":  
        soes1_capm_exc[col] = soes1_capm[col] - soes1_capm["rf"]

soes1_capm_exc = soes1_capm_exc.rename(columns=lambda x: f"exc_{x}" if x != "rf" else x)
soes1_capm_exc = soes1_capm_exc.drop(columns=["rf"])
soes1_capm_exc.to_csv("soes1_capm_exc.csv")

print(soes1_capm_exc)

            exc_000021.SZ  exc_000027.SZ  exc_000059.SZ  exc_000060.SZ  \
Date                                                                     
2012-02-29       0.121053       0.037979       0.077225       0.111843   
2012-03-31      -0.081446      -0.079745      -0.099394      -0.212129   
2012-04-30       0.103164       0.029829       0.079579       0.082086   
2012-05-31      -0.041960       0.045847      -0.052905      -0.008172   
2012-06-30      -0.115161      -0.010460      -0.105971      -0.089387   
...                   ...            ...            ...            ...   
2024-07-31      -0.051256      -0.080103      -0.018619      -0.011250   
2024-08-31      -0.048750      -0.136390      -0.036371      -0.001800   
2024-09-30       0.183813       0.167747       0.181378       0.164540   
2024-10-31       0.355148      -0.048629       0.114619      -0.009866   
2024-11-30      -0.160088      -0.009219      -0.024294      -0.016024   

            exc_000089.SZ  exc_000338

In [13]:
# non SOEs df
file_path = r"C:\Users\hp\Favorites\Desktop\Thesis\All Chinese non SOEs.xlsx"  
nsoes1 = pd.read_excel(file_path, usecols="A", engine="openpyxl")  
nsoes1.columns = ["Tickers"]

nsoes1_data_ek = []

for ticker in nsoes1["Tickers"]:
    try:
        print(f"Fetching data for {ticker}...")
        data = ek.get_timeseries(ticker, fields="CLOSE", start_date="2012-01-01", end_date="2025-01-01", interval="monthly")
        data['Ticker'] = ticker
        nsoes1_data_ek.append(data)
        
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
nsoes1_data_ = pd.concat(nsoes1_data_ek)
nsoes1_data = nsoes1_data_.pivot_table(index='Date', columns='Ticker', values='CLOSE')
nsoes1_data = nsoes1_data.dropna(axis=1)
rnsoes1 = np.log(nsoes1_data) - np.log(nsoes1_data.shift(1))
rnsoes1 = rnsoes1.dropna()
rnsoes1.columns.name = None

rnsoes1.to_csv("rnsoes1.csv")

Fetching data for 300750.SZ...
Fetching data for 600036.SS...
Fetching data for 002594.SZ...
Fetching data for 601318.SS...
Fetching data for 000333.SZ...
Fetching data for 601328.SS...
Fetching data for 601166.SS...
Fetching data for 601138.SS...
Fetching data for 600030.SS...
Fetching data for 300059.SZ...
Fetching data for 688041.SS...
Fetching data for 688256.SS...
Fetching data for 300760.SZ...
Fetching data for 600276.SS...
Fetching data for 002475.SZ...
Fetching data for 601601.SS...
Fetching data for 600690.SS...
Fetching data for 000651.SZ...
Fetching data for 000001.SZ...
Fetching data for 603288.SS...
Fetching data for 002352.SZ...
Fetching data for 002714.SZ...
Fetching data for 688235.SS...
Fetching data for 300124.SZ...
Fetching data for 601633.SS...
Fetching data for 601127.SS...
Fetching data for 600887.SS...
Fetching data for 603259.SS...
Fetching data for 600016.SS...
Fetching data for 300999.SZ...
Fetching data for 600031.SS...
Fetching data for 300033.SZ...
Fetching

2025-06-12 09:28:43,194 P[23016] [MainThread 6232] Error with 600811.SS: Invalid RIC
2025-06-12 09:28:43,196 P[23016] [MainThread 6232] 600811.SS: Invalid RIC | 


Error fetching data for 600811.SS: Error code -1 | 600811.SS: Invalid RIC | 
Fetching data for 603863.SS...
Fetching data for 000826.SZ...
Fetching data for 600804.SS...
Fetching data for 603377.SS...
Fetching data for 300594.SZ...
Fetching data for 300630.SZ...


2025-06-12 09:28:44,530 P[23016] [MainThread 6232] Error with 300630.SZ: Invalid RIC
2025-06-12 09:28:44,532 P[23016] [MainThread 6232] 300630.SZ: Invalid RIC | 


Error fetching data for 300630.SZ: Error code -1 | 300630.SZ: Invalid RIC | 


In [15]:
file_path = r"C:\Users\hp\Favorites\Desktop\Thesis\rnsoes1.csv"
rnsoes1 = pd.read_csv(file_path)
rnsoes1['Date'] = pd.to_datetime(rnsoes1['Date'])
rnsoes1.set_index('Date', inplace=True)

# Merge all the data (SOEs, Market, Risk-Free Rate)
nsoes1_capm = pd.merge(pd.merge(rnsoes1, rmkt, on="Date", how="inner"), rf, on="Date", how="inner")

nsoes1_capm_exc = nsoes1_capm.copy()
for col in nsoes1_capm.columns:
    if col != "rf":  
        nsoes1_capm_exc[col] = nsoes1_capm[col] - nsoes1_capm["rf"]

nsoes1_capm_exc = nsoes1_capm_exc.rename(columns=lambda x: f"exc_{x}" if x != "rf" else x)
nsoes1_capm_exc = nsoes1_capm_exc.drop(columns=["rf"])
nsoes1_capm_exc.to_csv("nsoes1_capm_exc.csv")

print(nsoes1_capm_exc)

            exc_000001.SZ  exc_000012.SZ  exc_000039.SZ  exc_000078.SZ  \
Date                                                                     
2012-02-29       0.024362       0.091667       0.083035       0.117650   
2012-03-31      -0.087688      -0.195223      -0.119955      -0.126594   
2012-04-30       0.049786       0.076449       0.127128       0.065250   
2012-05-31      -0.056833       0.013493      -0.033659       0.063683   
2012-06-30      -0.037125      -0.094024      -0.125046      -0.063472   
...                   ...            ...            ...            ...   
2024-07-31       0.009982      -0.001771      -0.156822       0.146974   
2024-08-31      -0.012568       0.008072      -0.056217      -0.091031   
2024-09-30       0.182016       0.125345       0.178097       0.150463   
2024-10-31      -0.072166      -0.049608      -0.042637       0.057508   
2024-11-30      -0.001694      -0.003511      -0.061378       0.037111   

            exc_000157.SZ  exc_000517